In [ ]:
# Whisper and audio tools
!pip install git+https://github.com/openai/whisper.git
!pip install pydub openai ffmpeg-python

# Set up FFmpeg
!apt install ffmpeg -y


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2gumf6pg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2gumf6pg
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.m

In [ ]:
from google.colab import files

uploaded = files.upload()
# This gives you something like 'your_audio.mp3'
filename = list(uploaded.keys())[0]
print("Uploaded:", filename)


Saving call_1.mp3 to call_1.mp3
Uploaded: call_1.mp3


In [ ]:
import whisper
import openai
from openai import OpenAI
import os
from pydub import AudioSegment
import datetime

apikey = "OPENAI_API_KEY"  # Replace with your OpenAI API key
client = OpenAI(api_key = apikey)

def split_audio(file_path, chunk_length_ms=10 * 60 * 1000):
    audio = AudioSegment.from_file(file_path)
    chunks = []
    for i in range(0, len(audio), chunk_length_ms):
        chunk = audio[i:i+chunk_length_ms]
        chunk_path = f"{file_path}_chunk_{i}.wav"
        chunk.export(chunk_path, format="wav")
        chunks.append(chunk_path)
    print(f"✅ Split into {len(chunks)} chunks")
    return chunks

def transcribe_audio_serial(file_path):
    model = whisper.load_model("medium")  # You can use 'small' or 'base' for speed
    chunks = split_audio(file_path)
    transcript_parts = []

    for idx, chunk_path in enumerate(chunks):
        print(f"🔊 Transcribing chunk {idx + 1} / {len(chunks)}...")
        result = model.transcribe(chunk_path, task="translate")
        transcript_parts.append(result['text'])
        os.remove(chunk_path)

    print("✅ All chunks done.")
    return " ".join(transcript_parts).strip()

def extract_info_from_transcript(transcript):
    prompt = f"""
You are a police assistant AI. Given the following transcript of a police complaint call, extract:
1. Complaint Category (e.g., Robbery, Assault, Cybercrime, etc.)
2. Location of Incident
3. Time or Date if mentioned
4. Any names of suspects or victims mentioned
5. Any other relevant details

Transcript:
\"\"\"{transcript}\"\"\"

Return the result as a JSON object with keys:
"category", "location", "time", "suspect_or_victim", "other_details"
"""
    response = client.responses.create(
        model="gpt-4",
        instructions="You are a police assistant that goes through audio files to find out details about the crime.",
        input=prompt
    )
    return response.output_text

# Run everything
transcript = transcribe_audio_serial(filename)
print("\n📝 Full Transcript:\n", transcript[:1000])  # Preview

info = extract_info_from_transcript(transcript)
print("\n📊 Extracted Info:\n", info)


100%|█████████████████████████████████████| 1.42G/1.42G [00:20<00:00, 74.1MiB/s]


✅ Split into 5 chunks
🔊 Transcribing chunk 1 / 5...
🔊 Transcribing chunk 2 / 5...
🔊 Transcribing chunk 3 / 5...
🔊 Transcribing chunk 4 / 5...
🔊 Transcribing chunk 5 / 5...
✅ All chunks done.

📝 Full Transcript:
 Officer, Tacoma 66, TNW, drive. I will be transporting victim to North Hollywood Medical Center. TNW, you're not going to be able to ring up 5026 Langisham Boulevard at New Wave Communications, incident 1464. 59, Officer, TNW, cancel the outside unit. You have a call. 15813, Roger. 989, 989, 839, cancel the call. Riverside Drive, 15813, we're handout. 989, Roger. 840, question assistance, we have a possible 211 in progress at the Bank of America. Bank of America. North of Kittredge, we have shots fired. 39, show us your responding code 3. All units, Officer, need help at the Bank of America, North of Kittredge. Officer, needs help at the Bank of America, North of Kittredge. 39, we're responding code 3. 15839 is responding code 3. 15839, officer, needs help, BMA. All units, Offi

In [ ]:
# Save transcript
with open("transcript.txt", "w") as f:
    f.write(transcript)

# Save extracted info
with open("info.json", "w") as f:
    f.write(info)

files.download("transcript.txt")
files.download("info.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>